In [21]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore')

In [2]:
from glob import glob
from tqdm import tqdm

In [3]:
path = 'data/'

sample = pd.read_csv(path + 'sample_submission.csv')
train = pd.read_csv(path + 'train_set.ReorgE.csv')
test = pd.read_csv(path + 'test_set.csv')

In [4]:
train

,index,SMILES,Reorg_g,Reorg_ex
0,train_0,CC[C@H]1CCCCN1C(=O)[C@@H](C)OC(=O)c1c(C)oc(-n2...,0.631486,0.535060
1,train_1,O[C@@H](CNC1CC1)CN1CCc2sccc2C1,0.825901,1.116781
2,train_2,N#CCCNC(=O)[C@@]1(O)CCSC1,1.463943,0.964848
3,train_3,COC[C@H]1CN(c2ccc(OCC[C@@H](C)O)cc2)C(=O)O1,0.166669,0.161458
4,train_4,N#Cc1c(-c2ccccc2OCC(N)=O)[nH]c(C(N)=O)c1N,0.313820,0.338862
...,...,...,...,...
18152,train_18152,CC(=O)Nc1ccc2ccc3cccc4ccc1c2c34,0.146917,0.143084
18153,train_18153,CC(C)(C)c1ccccc1N(c1ccccc1)c1ccc(S(=O)(=O)c2cc...,0.612898,0.500668
18154,train_18154,CN(C)c1ccc(C(=O)Nc2ccccc2)cc1,1.218777,1.048954
18155,train_18155,c1ccc(N(c2ccccc2)c2ccc(-c3ncc(-c4ccc(-c5cnc(-c...,0.145292,0.182589


## Loading MOL Files
- train_set
- test_set

### train_set

In [5]:
path = 'data/mol_files/train_set/'

train_mol = sorted(glob(path + '*.mol'))

In [6]:
train_mol[:10]

['data/mol_files/train_set\\train_0_ex.mol',
 'data/mol_files/train_set\\train_0_g.mol',
 'data/mol_files/train_set\\train_10000_ex.mol',
 'data/mol_files/train_set\\train_10000_g.mol',
 'data/mol_files/train_set\\train_10001_ex.mol',
 'data/mol_files/train_set\\train_10001_g.mol',
 'data/mol_files/train_set\\train_10002_ex.mol',
 'data/mol_files/train_set\\train_10002_g.mol',
 'data/mol_files/train_set\\train_10003_ex.mol',
 'data/mol_files/train_set\\train_10003_g.mol']

In [7]:
mol_file = open(train_mol[0], 'r').read().split("\n")
mol_file

##### Anatomy of a mol file #####

# Counts Block
# 53: number of atoms
# 55: number of bonds

# Atoms Block
# 3.0263   -2.3512    0.5507: x, y, z coordinates
# C: atom symbol

# Bonds Block(1  2  1  0)
# 1: first atom row number
# 2: second atom row number
# 1: bond type
# 0: bond stereochemistry

['',
 '     RDKit          3D',
 '',
 ' 53 55  0  0  0  0  0  0  0  0999 V2000',
 '    3.0263   -2.3512    0.5507 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    3.4398   -0.9997    1.1366 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    3.3570    0.1784    0.1392 C   0  0  1  0  0  0  0  0  0  0  0  0',
 '    1.9344    0.5133   -0.3583 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    1.0782    1.2336    0.6877 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    1.8074    2.4846    1.1839 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    3.2042    2.1347    1.6952 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    3.9824    1.3938    0.6950 N   0  0  0  0  0  0  0  0  0  0  0  0',
 '    5.3098    1.7286    0.5618 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    5.8180    2.6625    1.1588 O   0  0  0  0  0  0  0  0  0  0  0  0',
 '    6.1704    0.9343   -0.4511 C   0  0  1  0  0  0  0  0  0  0  0  0',
 '    6.1652    1.6104   -1.8172 C   0  0  0  0  0  0  0  0  0  0  0  0',
 '    7.5285    0.9327   -0.009

#### Length of an Atom

In [8]:
tmp = re.sub(' +', ' ', mol_file[4].lstrip()).split(' ')

print(tmp)
len(tmp)

['3.0263', '-2.3512', '0.5507', 'C', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


16

In [9]:
length = []

for i in mol_file:
    tmp = re.sub(' +', ' ', i.lstrip()).split(' ')
    length.append(len(tmp))

set(length)

# max length: 16

{1, 2, 4, 11, 16}

### test_set

In [10]:
path = 'data/mol_files/test_set/'

test_mol = sorted(glob(path + '*.mol'))

In [11]:
test_mol[:10]

['data/mol_files/test_set\\test_0_ex.mol',
 'data/mol_files/test_set\\test_0_g.mol',
 'data/mol_files/test_set\\test_100_ex.mol',
 'data/mol_files/test_set\\test_100_g.mol',
 'data/mol_files/test_set\\test_101_ex.mol',
 'data/mol_files/test_set\\test_101_g.mol',
 'data/mol_files/test_set\\test_102_ex.mol',
 'data/mol_files/test_set\\test_102_g.mol',
 'data/mol_files/test_set\\test_103_ex.mol',
 'data/mol_files/test_set\\test_103_g.mol']

## Creating CSV Files
- atoms block
    - x, y, z coordinates
    - atom symbol

### train_set

In [12]:
# 분자구조 데이터를 분석 가능한 데이터프레임 형태로 변환

for i in tqdm(train_mol):
    
    atoms = []    
    mol_file = open(i, 'r').read().split("\n")
    
    # 필요한 정보(atoms block)를 추출하여 리스트 형태로 저장
    for j in mol_file:
        tmp = re.sub(' +', ' ', j.lstrip()).split(' ') # 공백 제거
        
        if len(tmp) == 16: # atoms block(max length)에 해당하는 경우
            atoms.append(tmp) # nested list
            
    file_name = i.split('\\')[-1].split('.')[0] # 확장자 제외 파일명(ex: train_0_g)
    path = 'data/mol_files/train_set/'
    
    # 분자별 atoms block 정보를 csv 파일로 저장
    pd.DataFrame(atoms).to_csv(path + file_name + '.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████| 36314/36314 [02:46<00:00, 218.69it/s]


### test_set

In [13]:
for i in tqdm(test_mol):
    
    atoms = []    
    mol_file = open(i, 'r').read().split("\n")
    
    for j in mol_file:
        tmp = re.sub(' +', ' ', j.lstrip()).split(' ')
        
        if len(tmp) == 16:
            atoms.append(tmp)
            
    file_name = i.split('\\')[-1].split('.')[0]
    path = 'data/mol_files/test_set/'
    
    pd.DataFrame(atoms).to_csv(path + file_name + '.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████| 914/914 [00:01<00:00, 710.07it/s]


## Data Preparation
- determine max number of atoms
    - train_set
    - test_set
- gather data as DataFrame

### Max Length

#### train_set

In [56]:
path = 'data/mol_files/train_set/'
max_train = 0 # max number of atoms(rows)

for i in tqdm(train['index']):
    
    mol = pd.read_csv(path + i + '_g.csv')

    if len(mol) >= max_train:
        max_train = len(mol)

100%|██████████████████████████████████████████████████████████████████████████| 18157/18157 [00:15<00:00, 1161.94it/s]


#### test_set

In [57]:
path = 'data/mol_files/test_set/'
max_test = 0

for i in tqdm(test['index']):
    
    mol = pd.read_csv(path + i + '_g.csv')

    if len(mol) >= max_test:
        max_test = len(mol)

100%|██████████████████████████████████████████████████████████████████████████████| 457/457 [00:00<00:00, 1142.32it/s]


In [58]:
max_len = max(max_train, max_test)
max_len

310

### Gathering Data as DataFrame
- create DataFrame for modeling

#### train_set

In [34]:
path = 'data/mol_files/train_set/'

g_data, ex_data = [], []
zeros = [[0] * 3]

for i in tqdm(train['index']):
    
    g = pd.read_csv(path + i + '_g.csv').iloc[:, :3]
    ex = pd.read_csv(path + i +'_ex.csv').iloc[:, :3]
    
    g = np.array(g)
    ex = np.array(ex)
    
    if max_len == len(g):
        g, ex = g, ex
    else:
        g = np.concatenate((g, np.array(zeros * (max_len - len(g)))))
        ex = np.concatenate((ex, np.array(zeros * (max_len - len(ex)))))
    
    g_data.append(g)
    ex_data.append(ex)

100%|███████████████████████████████████████████████████████████████████████████| 18157/18157 [00:38<00:00, 477.64it/s]


In [60]:
print(max_len == len(g_data[0]))
print(max_len == len(g_data[-1]))

True
True


In [41]:
tmp_g = np.array(g_data).reshape(len(g_data), -1)
tmp_ex = np.array(ex_data).reshape(len(ex_data), -1)

In [44]:
tmp_g.shape, tmp_ex.shape

((18157, 930), (18157, 930))

In [83]:
df_g = pd.DataFrame(tmp_g)
df_g = pd.concat([df_g, train['Reorg_g']], axis=1)
df_g.head()

,0,1,2,3,4,5,6,7,8,9,...,921,922,923,924,925,926,927,928,929,Reorg_g
0,3.7175,1.3796,0.3069,2.6911,0.6650,0.8028,2.2271,1.3412,1.8738,1.7083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.631486
1,1.0674,0.0735,0.0396,2.5287,0.0889,0.0481,3.1825,1.3425,0.0516,2.5753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.825901
2,1.8910,0.7359,0.6172,2.6962,1.2480,-0.4256,3.1597,0.4038,-1.3611,3.9113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.463943
3,6.2568,-5.4248,11.5729,7.3968,-5.6295,10.5743,6.9215,-5.8281,9.1316,8.0577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166669
4,6.2542,4.5402,2.4520,6.3931,3.3969,2.2861,6.5676,2.0058,2.0830,5.6336,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313820


In [82]:
df_ex = pd.DataFrame(tmp_ex)
df_ex = pd.concat([df_ex, train['Reorg_ex']], axis=1)
df_ex.head()

,0,1,2,3,4,5,6,7,8,9,...,921,922,923,924,925,926,927,928,929,Reorg_ex
0,3.8129,1.5265,0.9334,2.7799,0.6656,0.8951,1.8536,1.1313,1.7603,2.2322,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.535060
1,1.0492,0.0682,0.0394,2.5071,0.0643,0.0478,3.1892,1.3624,0.0519,2.5386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.116781
2,1.6107,1.1188,0.3498,2.9401,1.1829,-0.1400,3.2440,0.3934,-1.1928,3.8552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.964848
3,6.2433,-5.5782,11.7891,7.3745,-5.7728,10.7786,6.8868,-5.9404,9.3360,8.0138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.161458
4,6.1247,4.5023,2.1345,6.3000,3.3524,2.0231,6.5153,1.9688,1.8884,5.5734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.338862


In [59]:
df_g.to_csv('train_g.csv', index=False)
df_ex.to_csv('train_ex.csv', index=False)

#### test_set

In [61]:
path = 'data/mol_files/test_set/'

g_data, ex_data = [], []
zeros = [[0] * 3]

for i in tqdm(test['index']):
    
    g = pd.read_csv(path + i + '_g.csv').iloc[:, :3]
    ex = pd.read_csv(path + i +'_ex.csv').iloc[:, :3]
    
    g = np.array(g)
    ex = np.array(ex)
    
    if max_len == len(g):
        g, ex = g, ex
    else:
        g = np.concatenate((g, np.array(zeros * (max_len - len(g)))))
        ex = np.concatenate((ex, np.array(zeros * (max_len - len(ex)))))
    
    g_data.append(g)
    ex_data.append(ex)

100%|███████████████████████████████████████████████████████████████████████████████| 457/457 [00:02<00:00, 201.01it/s]


In [62]:
print(max_len == len(g_data[0]))
print(max_len == len(g_data[-1]))

True
True


In [63]:
tmp_g = np.array(g_data).reshape(len(g_data), -1)
tmp_ex = np.array(ex_data).reshape(len(ex_data), -1)

In [64]:
tmp_g.shape, tmp_ex.shape

((457, 930), (457, 930))

In [80]:
df_g = pd.DataFrame(tmp_g)
df_g.head()

,0,1,2,3,4,5,6,7,8,9,...,920,921,922,923,924,925,926,927,928,929
0,3.7175,1.3796,0.3069,2.6911,0.6650,0.8028,2.2271,1.3412,1.8738,1.7083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0674,0.0735,0.0396,2.5287,0.0889,0.0481,3.1825,1.3425,0.0516,2.5753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.8910,0.7359,0.6172,2.6962,1.2480,-0.4256,3.1597,0.4038,-1.3611,3.9113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.2568,-5.4248,11.5729,7.3968,-5.6295,10.5743,6.9215,-5.8281,9.1316,8.0577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.2542,4.5402,2.4520,6.3931,3.3969,2.2861,6.5676,2.0058,2.0830,5.6336,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
df_ex = pd.DataFrame(tmp_ex)
df_ex.head()

,0,1,2,3,4,5,6,7,8,9,...,920,921,922,923,924,925,926,927,928,929
0,3.8129,1.5265,0.9334,2.7799,0.6656,0.8951,1.8536,1.1313,1.7603,2.2322,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0492,0.0682,0.0394,2.5071,0.0643,0.0478,3.1892,1.3624,0.0519,2.5386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.6107,1.1188,0.3498,2.9401,1.1829,-0.1400,3.2440,0.3934,-1.1928,3.8552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.2433,-5.5782,11.7891,7.3745,-5.7728,10.7786,6.8868,-5.9404,9.3360,8.0138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.1247,4.5023,2.1345,6.3000,3.3524,2.0231,6.5153,1.9688,1.8884,5.5734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df_g.to_csv('test_g.csv', index=False)
df_ex.to_csv('test_ex.csv', index=False)

## Modeling

#### train_g, train_ex

In [69]:
train_g = pd.read_csv('train_g.csv')
train_ex = pd.read_csv('train_ex.csv')

In [73]:
train_g['Reorg'] = train_g['Reorg_g']
train_ex['Reorg'] = train_ex['Reorg_ex']

In [77]:
train_g = train_g.drop('Reorg_g', axis=1)
train_ex = train_ex.drop('Reorg_ex', axis=1)

In [79]:
train_set = pd.concat([train_g, train_ex], ignore_index=True)
train_set

,0,1,2,3,4,5,6,7,8,9,...,921,922,923,924,925,926,927,928,929,Reorg
0,2.8972,-2.3256,0.4309,3.3467,-1.0151,1.0796,3.3365,0.2041,0.1293,1.9424,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.631486
1,2.2446,2.0206,-0.4231,2.0725,0.6141,-0.5399,0.5932,0.2964,-0.8063,0.3246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.825901
2,4.8695,-1.9497,-5.1308,4.3832,-2.2384,-4.1192,3.8213,-2.6331,-2.8261,4.5197,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.463943
3,5.0694,-5.7034,0.2680,5.3278,-4.4707,0.8913,4.5156,-3.4187,0.4424,4.9288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166669
4,0.9331,-0.6287,-0.7864,2.0350,-0.3830,-0.5019,3.3942,-0.1770,-0.1553,4.1081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.313820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36309,1.0234,-1.4846,0.8680,2.4267,-1.9944,0.5913,2.6610,-3.1658,0.3641,3.4096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143084
36310,0.0606,-4.9574,-0.9820,-1.2920,-5.3186,-1.6425,-1.0078,-6.4309,-2.6791,-2.2422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500668
36311,2.0764,0.1408,1.1949,3.0072,0.2949,0.1004,4.1022,-0.6436,0.0294,3.1707,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.048954
36312,11.5467,1.7131,6.0287,11.9437,2.2432,4.7961,12.1085,1.4132,3.6872,11.8885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.182589


#### test_g, test_ex

In [85]:
test_g = pd.read_csv('test_g.csv')
test_ex = pd.read_csv('test_ex.csv')

In [86]:
test_set = pd.concat([test_g, test_ex], ignore_index=True)
test_set

,0,1,2,3,4,5,6,7,8,9,...,920,921,922,923,924,925,926,927,928,929
0,3.7175,1.3796,0.3069,2.6911,0.6650,0.8028,2.2271,1.3412,1.8738,1.7083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0674,0.0735,0.0396,2.5287,0.0889,0.0481,3.1825,1.3425,0.0516,2.5753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.8910,0.7359,0.6172,2.6962,1.2480,-0.4256,3.1597,0.4038,-1.3611,3.9113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.2568,-5.4248,11.5729,7.3968,-5.6295,10.5743,6.9215,-5.8281,9.1316,8.0577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6.2542,4.5402,2.4520,6.3931,3.3969,2.2861,6.5676,2.0058,2.0830,5.6336,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,2.8301,3.3614,-7.3273,2.1012,3.0747,-6.4626,1.2470,2.7029,-5.3896,1.7672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,-9.0739,-2.9812,1.1936,-8.0225,-3.5556,0.2073,-8.7051,-4.5928,-0.7245,-6.9132,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
911,-3.1227,2.5263,-0.1338,-3.8498,1.2034,-0.1167,-5.2491,1.1887,-0.1493,-5.9349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
912,2.9762,-2.5958,0.8796,3.7765,-1.3720,0.5079,5.1715,-1.4187,0.5654,5.9272,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### modeling

In [87]:
from xgboost import XGBRegressor as xg
from catboost import CatBoostRegressor as cb
from lightgbm import LGBMRegressor as lg
from sklearn.ensemble import RandomForestRegressor as rf

In [93]:
X_train = train_set.iloc[:, :-1]
y_train = train_set['Reorg']

In [94]:
X_test = test_set

In [95]:
model = rf(n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [96]:
model.score(X_train, y_train)

0.9220878664157834

In [115]:
pred = model.predict(X_test)

In [116]:
sample['Reorg_g'] = pred[:457]
sample['Reorg_ex'] = pred[457:]
sample

,index,Reorg_g,Reorg_ex
0,test_0,0.606825,0.601911
1,test_1,0.425512,0.424709
2,test_2,0.610753,0.626665
3,test_3,0.679628,0.671300
4,test_4,0.621258,0.605396
...,...,...,...
452,test_452,0.618677,0.667166
453,test_453,0.472977,0.468113
454,test_454,0.411507,0.408763
455,test_455,0.559498,0.565408


In [117]:
sample.to_csv('submission_1.csv', index=False)